In [1]:
import pandas as pd

In [4]:
suscripciones_o = pd.read_excel('raw_data/paresv2.xlsx', sheet_name='subscripciones')
generos_o = pd.read_excel('raw_data/paresv2.xlsx', sheet_name='genero')
pagos_o = pd.read_excel('raw_data/paresv2.xlsx', sheet_name='pagos')
proveedores_o = pd.read_excel('raw_data/paresv2.xlsx', sheet_name='proveedores')
usuario_proveedor_o = pd.read_excel('raw_data/paresv2.xlsx', sheet_name='usuario proveedor')
usuario_actividad_o = pd.read_excel('raw_data/paresv2.xlsx', sheet_name='usuario actividades')
videojuego_genero_o = pd.read_excel('raw_data/paresv2.xlsx', sheet_name='videojuego genero')
videojuego_o = pd.read_excel('raw_data/paresv2.xlsx', sheet_name='videojuego')

In [6]:
# Videojuegos (id_videojuego, titulo, puntuacion, clasificacion, fecha, beneficio_preorden)
select = videojuego_o[["id_videojuego", "titulo", "puntuacion", "clasificacion", "fecha_de_lanzamiento", "beneficio_preorden"]]
select.drop_duplicates(subset="id_videojuego", keep="first", inplace=True)
select.to_csv("filtered_data/videojuegos.csv", index=False)

/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/2886203505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select.drop_duplicates(subset="id_videojuego", keep="first", inplace=True)


In [8]:
# Usuarios (id_usuario, nombre, username, mail, password, fecha_nacimiento)
select = usuario_actividad_o[["id_usuario", "nombre", "username", "mail", "password", "fecha_nacimiento"]]
select.drop_duplicates(subset="id_usuario", keep="first", inplace=True)
select.to_csv("filtered_data/usuarios.csv", index=False)

/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/2146107642.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select.drop_duplicates(subset="id_usuario", keep="first", inplace=True)


In [13]:
# Proveedores (id_proveedor, nombre, plataforma)
select = proveedores_o[["id", "nombre", "plataforma"]]
select.drop_duplicates(subset="id", keep="first", inplace=True)
select.to_csv("filtered_data/proveedoresvj.csv", index=False)

/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/3086117897.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select.drop_duplicates(subset="id", keep="first", inplace=True)


In [16]:
# Resenas (id_resena, titulo, texto, veredicto)
resena_con_user_y_vj = usuario_actividad_o[["id_usuario","id_videojuego","titulo","texto","veredicto"]]
filter_condition = resena_con_user_y_vj[["veredicto", "titulo", "texto"]].notna().all(axis=1)
resena_con_user_y_vj = resena_con_user_y_vj[filter_condition]
resena_con_user_y_vj['id_resena'] = range(1, len(resena_con_user_y_vj) + 1)
select = resena_con_user_y_vj[["id_resena", "titulo", "texto", "veredicto"]]
select.to_csv("filtered_data/resenas.csv", index=False)

In [43]:
# Pagos(pago_id, monto, fecha_pago, id_usuario, id_videojuego)
# Pagos (pago_id, monto, fecha, id_usuario, id_videojuego)
subs_act_o = suscripciones_o[suscripciones_o["estado"] == "active"]
subs_ina_o = suscripciones_o[suscripciones_o["estado"] == "canceled"]
subs_id_vj = suscripciones_o[["id", "id_videojuego"]].rename(columns={"id": "subs_id"})

pagos_sin_sub = pagos_o[pagos_o["subs_id"].isnull()]
pagos_con_sub = pd.concat([pagos_o, pagos_sin_sub]).drop_duplicates(keep=False)

pagos_sin_sub["id_proveedor"] = pagos_sin_sub["id_proveedor"].astype(int)
pagos_sin_sub["id_videojuego"] = pagos_sin_sub["id_videojuego"].astype(int)
pagos_con_sub["subs_id"] = pagos_con_sub["subs_id"].astype(int)

# pagos 1 + pagos 2(le falta id_vj) = pagos
pagos_1 = pagos_sin_sub[["pago_id", "monto", "fecha", "id_usuario", "id_videojuego"]]
pagos_2 = pagos_con_sub[["pago_id", "monto", "fecha", "id_usuario", "subs_id"]]

# Using subs_id_vj to get id_videojuego in pagos_2
pagos_2_con_sub_vj = pd.merge(pagos_2, subs_id_vj, on="subs_id", how="left")
# now i delete the subs_id column in pagos_2
pagos_22 = pagos_2_con_sub_vj.drop(columns=["subs_id"])

# now i add the rows of pagos_1 and pagos_22
pagos = pd.concat([pagos_1, pagos_22])
pagos.to_csv("filtered_data/pagos.csv", index=False)

/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/2265926362.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagos_sin_sub["id_proveedor"] = pagos_sin_sub["id_proveedor"].astype(int)
/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/2265926362.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pagos_sin_sub["id_videojuego"] = pagos_sin_sub["id_videojuego"].astype(int)


In [45]:
# Pago_unico(pago_id, id_proveedor, preorden)
pago_unico = pagos_sin_sub[['pago_id', 'id_proveedor', 'preorden']]
pago_unico.to_csv("filtered_data/pago_unico.csv", index=False)

In [46]:
# Suscripciones(pago_id, id_suscripcion, fecha_inicio, mensualidad)
id_e_f_m = suscripciones_o[["id", "estado", "fecha_inicio", "mensualidad"]]
id_e_f_m.rename(columns={"id": "subs_id"}, inplace=True)
# Falta agregar el pago_id correspondiente a cada id_suscripcion hint:usar pagos_2_con_sub_vj
# id_suscripcion, estado, fecha_inicio, mensualidad, pago_id
suscripciones_ai = pd.merge(id_e_f_m, pagos_2_con_sub_vj, on="subs_id", how="inner")
suscripciones = suscripciones_ai.drop(columns=["estado","monto", "fecha", "id_usuario", "id_videojuego"])
suscripciones.rename(columns={"subs_id": "id_suscripcion"}, inplace=True)
suscripciones = suscripciones[["pago_id", "id_suscripcion", "fecha_inicio", "mensualidad"]]
suscripciones.to_csv("filtered_data/suscripciones.csv", index=False)

/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/1907337426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_e_f_m.rename(columns={"id": "subs_id"}, inplace=True)


In [47]:
# Activa(pago_id, id_suscripcion)
activas = suscripciones_ai[suscripciones_ai["estado"] == "active"]
activas = activas[["pago_id", "subs_id"]]
activas.rename(columns={"subs_id": "id_suscripcion"}, inplace=True)
activas.to_csv("filtered_data/activas.csv", index=False)

In [48]:
# Inactiva(pago_id, id_suscripcion, fecha_termino)
aux = suscripciones_o[["id", "estado", "fecha_termino"]]
aux.rename(columns={"id": "subs_id"}, inplace=True)
suscripciones_ai = pd.merge(aux, pagos_2_con_sub_vj, on="subs_id", how="inner")
inactivas = suscripciones_ai[suscripciones_ai["estado"] == "canceled"]
inactivas = inactivas[["pago_id", "subs_id", "fecha_termino"]]
inactivas.rename(columns={"subs_id": "id_suscripcion"}, inplace=True)
inactivas.to_csv("filtered_data/inactivas.csv", index=False)

/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/3314481865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux.rename(columns={"id": "subs_id"}, inplace=True)


In [50]:
pagos_o.head()

,pago_id,monto,fecha,id_usuario,preorden,id_proveedor,id_videojuego,subs_id
0,4643,9100,2020-05-12,106,False,3.0,15.0,NaN
1,5081,9100,2021-06-20,82,False,3.0,15.0,NaN
2,5109,9100,2021-01-05,83,False,3.0,15.0,NaN
3,3961,9100,2021-03-03,125,False,3.0,15.0,NaN
4,4024,9100,2020-10-16,89,False,3.0,15.0,NaN


In [57]:
# Transaccion(pago_id, id_videojuego, id_usuario, id_proveedor)
transacciones = pagos[["pago_id", "id_videojuego", "id_usuario"]]

transacciones_con_proveedor = pd.merge(transacciones, pagos_o, on="pago_id", how="left")
transacciones_con_proveedor.drop(columns=["monto", "fecha", "id_usuario_y","preorden", "id_videojuego_y", "id_videojuego_y", "subs_id"], inplace=True)
transacciones_con_proveedor.rename(columns={"id_usuario_x": "id_usuario", "id_videojuego_x": "id_videojuego"}, inplace=True)
# if id_proveedor is null then it is a suscription and replace with 0
transacciones_con_proveedor["id_proveedor"] = transacciones_con_proveedor["id_proveedor"].fillna(0)
transacciones_con_proveedor["id_proveedor"] = transacciones_con_proveedor["id_proveedor"].astype(int)

transacciones_con_proveedor.to_csv("filtered_data/transacciones.csv", index=False)

In [31]:
# Horas_juego(id_usuario, id_videojuego, cantidad, fecha_visualizacion)
# Aqui asumimos que si un usuario no ha jugado un juego tiene cantidad = 0
select = usuario_actividad_o[["id_usuario", "id_videojuego", "cantidad", "fecha v"]]
hj = select[select["id_videojuego"].notnull()]
hj["id_videojuego"] = hj["id_videojuego"].astype(int)
# delete repeated rows with "id_usuario", "id_videojuego" and keep the first one
hj = hj.drop_duplicates(subset=["id_usuario", "id_videojuego"], keep="first")
# if id_videojuego not in videojuegos.csv then delete that row
hj = hj[hj["id_videojuego"].isin(videojuego_o["id_videojuego"])]
hj.to_csv("filtered_data/horas_juego.csv", index=False)

/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/1184528872.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hj["id_videojuego"] = hj["id_videojuego"].astype(int)


In [32]:
# Ofrece(id_proveedor, id_videojuego, precio, precio_preorden)
# Ojo con los nulos
select = proveedores_o[["id", "id_videojuego", "precio", "precio_preorden"]]
select.rename(columns={"id": "id_proveedor"}, inplace=True)
select.to_csv("filtered_data/ofrece.csv", index=False)

/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/1707776398.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select.rename(columns={"id": "id_proveedor"}, inplace=True)


In [33]:
# Opinion(id_resena, id_usuario, id_videojuego)
select = resena_con_user_y_vj[["id_resena", "id_usuario", "id_videojuego"]]
select["id_videojuego"] = select["id_videojuego"].astype(int)
# delete rows that have id_videojuego not in videojuegos.csv
select = select[select["id_videojuego"].isin(videojuego_o["id_videojuego"])]
select.to_csv("filtered_data/opinion.csv", index=False)

/var/folders/w_/hg_83yp12fxctnr04jpgsgg00000gn/T/ipykernel_7049/1768845900.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select["id_videojuego"] = select["id_videojuego"].astype(int)


In [40]:
# Tiene_cuenta(id_usuario, id_proveedor)
select = usuario_proveedor_o[["id_usuario", "id_proveedor"]]
select.drop_duplicates(subset=["id_usuario", "id_proveedor"], keep="first", inplace=True)
select.to_csv("filtered_data/tiene_cuenta.csv", index=False)

In [63]:
# Vj_genero(id_videojuego, genero)
select = videojuego_genero_o
select.to_csv("filtered_data/vj_genero.csv", index=False)

In [60]:
# Genero_subgenero(genero, subgenero)
genero_subgenero = generos_o[["genero", "subgenero"]].fillna("No tiene")
genero_subgenero.to_csv("filtered_data/genero_subgenero.csv", index=False)